In [1]:
import openpyxl
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing  import LabelEncoder, OneHotEncoder
from datetime import datetime



In [2]:
# --데이터 전처리
# 업체규모(EMP_CNT → 결측값 존재)
# 업력(ESTB_DATE → 결측값 존재)
# 매출액 – 상승추세
# 상장,비상장
# 휴폐업 이력 존재 유무

# 외부데이터
# 산업동향 – 산업위험
# ->통계청 업종별 경기 전망
# 기업 입지 - 도심지역, 외곽지역 (bizno 사업자등록번호 조회)
# (시장위험 – 경기,금리,환율,유가)

#KDD 데이터 분석 방법론 참고

# 데이터 가져오기

In [2]:
wb1 = openpyxl.load_workbook('./numble_data/휴폐업 중소법인 재무보유.xlsx')
wb2 = openpyxl.load_workbook('./numble_data/액티브 중소법인 재무보유.xlsx', read_only=True)

In [3]:
sheet_rest = wb1["기업개요"]
sheet_rest_record=wb1['휴폐업이력']

In [4]:
#(휴폐업) 총 6739개
df_rest = pd.DataFrame(sheet_rest.values)
df_rest = df_rest.replace('','NaN')
df_rest.columns = df_rest.iloc[0,:]
df_rest = df_rest.iloc[1:, :]
df_rest.isnull().sum()
df_rest = df_rest.fillna('NaN')

df_rest = df_rest.reset_index(drop=True)
df_rest.head()

,BIZ_NO,CMP_PFIX_NM,CMP_NM,CMP_SFIX_NM,CMP_ENM,BZ_TYP,CMP_SCL,PBCO_GB,PSN_CORP_GB,HDOF_BR_GB,...,MDSCO_PRTC_YN,HDOF_CMP_CD,ESTB_DATE,ESTB_GB,LIST_DATE,MN_BIZ_CONT,NATN_NM,HOMEPAGE_URL,CEO_NM,EMP_CNT
0,1268145633,(주),용진테크,NaN,"Yongjin Tech Co., Ltd.",M,2,2,1,1,...,N,NaN,20000529,01,NaN,자동차부품 제조판매업,NaN,www.yongjintech.com,안상동,40
1,1388101749,NaN,신구건설,(주),"SHINKU C0NSTRUCTION CO.,LTD.",M,2,2,1,1,...,N,NaN,19850110,01,NaN,건축자재 판매업,NaN,www.shinku.co.kr,소재필,13
2,1068179240,(주),석정미디어,NaN,sung jung,M,2,2,1,1,...,N,NaN,19990601,01,NaN,"토너, 잉크, 프린터 외 컴퓨터 관련 주변기기",NaN,NaN,이상호,1
3,2248114507,(주),새한실업,NaN,"SAEHAN INDUSTRY CO.,LTD.",M,2,2,1,1,...,N,NaN,19980522,01,NaN,금속 캔 및 기타 포장용기 제조업,NaN,NaN,김정남외 1명,6
4,1168132785,(주),석영엔터프라이즈,NaN,"Seokyoung Enterprise Co.,Ltd.",M,2,2,1,1,...,N,NaN,19930226,01,NaN,스마트트레이 외,NaN,www.systeel.co.kr,박성진,40


In [5]:
#(휴폐업이력)데이터 총 6840개 - 여러번 폐업한 기업 찾아보기
df_rest_record = pd.DataFrame(sheet_rest_record.values)
df_rest_record = df_rest_record.fillna('NaN')
df_rest_record.columns = df_rest_record.iloc[0,:]
df_rest_record = df_rest_record.iloc[1:, :]
df_rest_record = df_rest_record.reset_index(drop=True)
df_rest_record.head()

,BIZ_NO,END_DATE,STRT_DATE,CLSBZ_GB,STAT_OCR_DATE
0,1268145633,99991231,20220518,3,20220518
1,1388101749,99991231,20210319,3,20210319
2,1068179240,99991231,20200731,3,20191213
3,2248114507,99991231,20210531,3,20210531
4,1168132785,99991231,20210630,3,20210630


In [6]:
sheet_active = wb2['기업개요_외감']
sheet_active_rest = wb2['휴폐업이력']

In [7]:
#(액티브) 외감기업
df_active= pd.DataFrame(sheet_active.values)
df_active = df_active.replace('','NaN')
df_active = df_active.fillna('NaN')
df_active.columns = df_active.iloc[0, :]
df_active = df_active.iloc[1:, :]
df_active = df_active.reset_index(drop=True)
df_active.head()

,BIZ_NO,CMP_PFIX_NM,CMP_NM1,CMP_SFIX_NM,CMP_ENM,BZ_TYP,CMP_SCL,PBCO_GB,PSN_CORP_GB,HDOF_BR_GB,...,MDSCO_PRTC_YN,HDOF_CMP_CD,ESTB_DATE,ESTB_GB,LIST_DATE,MN_BIZ_CONT,NATN_NM,HOMEPAGE_URL,CEO_NM,EMP_CNT
0,1138111739,NaN,한일가스산업,(주),"HANIL GAS IND. CO.,LTD.",M,2,2,1,1,...,N,NaN,19660419,01,NaN,LP Gas 충전 및 판매업,NaN,www.hanilgas.com,홍순철외 1명,43
1,2078132193,(주),엠피아이,NaN,"EMPIALEE CO.,LTD.",M,2,2,1,1,...,N,NaN,19910806,01,NaN,가죽 및 대용가죽제품 제조업,NaN,www.mastercnd.co.kr,이스테판상수외 1명,35
2,1318167565,NaN,도영운수,(주),"DOYOUNG TRANSPORTATION CO.,LTD.",M,2,2,1,1,...,N,NaN,20011030,01,NaN,시내버스업,NaN,transport@naver.com,김명화,149
3,5048128251,(주),중원산업,NaN,"JOONGWON INDUSTRIAL CO.,LTD.",M,2,2,1,1,...,N,NaN,19990112,01,NaN,자동차 부품 제조,NaN,www.koreajw.com,김명구,75
4,1098164260,(주),카타나골프,NaN,"CATANA GOLF CO.,LTD.",M,2,2,1,1,...,N,NaN,20000531,01,NaN,골프용품 전반의 제조 및 수출업,NaN,NaN,김홍득,3


In [8]:
df_rest.isnull().sum().sum()

0

In [9]:
#(액티브) 휴폐업이력 - 4828개
df_active_record = pd.DataFrame(sheet_active_rest.values)
df_active_record = df_active_record.fillna('NaN')
df_active_record.columns = df_active_record.iloc[0, :]
df_active_record = df_active_record.iloc[1:, :]
df_active_record = df_active_record.reset_index(drop=True)
df_active_record.head()

,BIZ_NO,END_DATE,STRT_DATE,CLSBZ_GB,STAT_OCR_DATE
0,5088115592,99991231,20150209,5,NaN
1,5088115592,20150208,20200731,2,NaN
2,1138127100,99991231,20121105,5,NaN
3,1138127100,20121104,20200731,2,NaN
4,1138127100,20121004,20200731,5,NaN


In [10]:
# 액티브
# -액티브 외감 df_active_big 28982 
# -휴폐업이력 4828개
# -(액티브 df_active 198651) - 참고자료

# 휴폐업
# -기업개요 - 휴폐업 데이터 df_rest 6739
# -휴폐업이력  df_rest_record 6840 (여러번 휴폐업한 기업 찾기)

# EDA

In [11]:
#예측을 한다, 액티브+휴폐업 기업 섞여있을 때 폐업인지 아닌지 구분을 하는 것.
#일단 예측을 해보고 외부데이터를 추가하면서 성능 높아지는지 체크해야할듯

#액티브 데이터 n개, 휴폐업 데이터 n개 -> x값
#폐업:1, 정상영업:0 으로 정답y데이터 만들기
#문자열 데이터 인코딩하기


In [12]:
#휴폐업데이터에서 산업코드1로 groupby하기 - 그룹 총 1037개(너무많음)
#업종(M,O,V)별로 예측따로하기?
df1 = df_rest.groupby('BZ_TYP').count()
df1

,BIZ_NO,CMP_PFIX_NM,CMP_NM,CMP_SFIX_NM,CMP_ENM,CMP_SCL,PBCO_GB,PSN_CORP_GB,HDOF_BR_GB,FR_IVST_CORP_YN,...,MDSCO_PRTC_YN,HDOF_CMP_CD,ESTB_DATE,ESTB_GB,LIST_DATE,MN_BIZ_CONT,NATN_NM,HOMEPAGE_URL,CEO_NM,EMP_CNT
BZ_TYP,,,,,,,,,,,,,,,,,,,,,
M,6698,6698,6698,6698,6698,6698,6698,6698,6698,6698,...,6698,6698,6698,6698,6698,6698,6698,6698,6698,6698
O,38,38,38,38,38,38,38,38,38,38,...,38,38,38,38,38,38,38,38,38,38
V,3,3,3,3,3,3,3,3,3,3,...,3,3,3,3,3,3,3,3,3,3


In [133]:
df3 = df_active.groupby('IND_CD_ORDR').count()
df3

,BIZ_NO,CMP_PFIX_NM,CMP_NM1,CMP_SFIX_NM,CMP_ENM,BZ_TYP,CMP_SCL,PBCO_GB,PSN_CORP_GB,HDOF_BR_GB,...,MDSCO_PRTC_YN,HDOF_CMP_CD,ESTB_DATE,ESTB_GB,LIST_DATE,MN_BIZ_CONT,NATN_NM,HOMEPAGE_URL,CEO_NM,EMP_CNT
IND_CD_ORDR,,,,,,,,,,,,,,,,,,,,,
10,28972,28972,28972,28972,28972,28972,28972,28972,28972,28972,...,28972,28972,28972,28972,28972,28972,28972,28972,28972,28972
NaN,10,10,10,10,10,10,10,10,10,10,...,10,10,10,10,10,10,10,10,10,10


In [14]:
df_test=df_active.loc[df_active['IND_CD_ORDR']=='  ']
df_test

,BIZ_NO,CMP_PFIX_NM,CMP_NM1,CMP_SFIX_NM,CMP_ENM,BZ_TYP,CMP_SCL,PBCO_GB,PSN_CORP_GB,HDOF_BR_GB,...,MDSCO_PRTC_YN,HDOF_CMP_CD,ESTB_DATE,ESTB_GB,LIST_DATE,MN_BIZ_CONT,NATN_NM,HOMEPAGE_URL,CEO_NM,EMP_CNT


In [16]:
#결측치 확인
#휴폐업 이력이 있는 회사 df_rest_record 총 6739개
#휴폐업 데이터 df_rest 6840
#액티브 외감 df_active_big 28982 
#((액티브 df_active 198651))

df_active.isnull().sum().sum()


0

In [17]:
# 사업자번호	결산년월	유동자산	매출채권	비유동자산	유형자산	자산총계	유동부채
# 비유동부채	부  채  총  계	자본금	이익잉여금(결손금）	자본총계	매출액	판매비와관리비
# 영업이익（손실）	법인세비용차감전순손익	법인세비용	당기순이익(손실)	기업순이익률(%)
# 유보액/총자산(%)	유보액/납입자본(%)	매출액총이익률(%)	매출액영업이익률(%)	매출액순이익률(%)
# 수지비율(%)	경상수지비율	영업비율(%)	금융비용대매출액비율(%	금융비용대부채비율(%)
# 금융비용대총비용비율(%	부채비율(%)	차입금의존도(%)	자기자본비율(%)	순운전자본비율(%)	
# 유동부채비율(%)	비유동부채비율(%)	부채총계대 매출액(%)	총자본회전율(회)	재고자산회전율(회)
# 매출채권회전율(회)	매입채무회전율(회)	미수금	매출원가	무형자산	재고자산

## 전처리 시작

In [11]:
#사용할 컬럼들
select_columns = ['BIZ_NO', 'BZ_TYP','NATN_NM', 'PBCO_GB', 'PSN_CORP_GB', 'HDOF_BR_GB', 'FR_IVST_CORP_YN', 'VENT_YN', 'LIST_CD', 'IND_CD_ORDR', 'IND_CD1', 'IND_CD2', 'IND_CD3', 'MDSCO_PRTC_YN', 'HDOF_CMP_CD', 'ESTB_DATE', 'ESTB_GB', 'LIST_DATE', 'EMP_CNT']

In [12]:
len(select_columns)

19

In [13]:
df_rest_select = df_rest.loc[:, select_columns]
df_active_select = df_active.loc[:, select_columns]
df_active_select

,BIZ_NO,BZ_TYP,NATN_NM,PBCO_GB,PSN_CORP_GB,HDOF_BR_GB,FR_IVST_CORP_YN,VENT_YN,LIST_CD,IND_CD_ORDR,IND_CD1,IND_CD2,IND_CD3,MDSCO_PRTC_YN,HDOF_CMP_CD,ESTB_DATE,ESTB_GB,LIST_DATE,EMP_CNT
0,1138111739,M,NaN,2,1,1,N,N,NaN,10,47712,NaN,NaN,N,NaN,19660419,01,NaN,43
1,2078132193,M,NaN,2,1,1,N,N,NaN,10,15190,NaN,NaN,N,NaN,19910806,01,NaN,35
2,1318167565,M,NaN,2,1,1,N,N,NaN,10,49212,NaN,NaN,N,NaN,20011030,01,NaN,149
3,5048128251,M,NaN,2,1,1,N,N,NaN,10,30391,NaN,NaN,N,NaN,19990112,01,NaN,75
4,1098164260,M,NaN,2,1,1,N,N,NaN,10,46800,NaN,NaN,N,NaN,20000531,01,NaN,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28977,8048101453,M,NaN,2,1,1,N,N,NaN,NaN,NaN,NaN,NaN,N,NaN,20191206,01,NaN,0
28978,8028701430,M,NaN,2,1,1,N,N,NaN,10,68122,NaN,NaN,N,NaN,20200402,01,NaN,1
28979,4268501421,M,NaN,2,1,2,N,N,NaN,10,72100,NaN,NaN,N,1220726,19900108,01,NaN,81
28980,6518502462,M,NaN,2,1,2,N,N,NaN,10,23322,NaN,NaN,N,1583116,20220615,01,NaN,26


In [14]:
df_rest_select.isnull().sum().sum()

0

In [15]:
#라벨인코딩
#  ['FR_IVST_CORP_YN' - N이 0
#   'VENT_YN' - N이 0
#   'MDSCO_PRTC_YN'] N이 0
# 'NATN_NM' 국가 총 64개

In [16]:
encode_features = ['FR_IVST_CORP_YN' , 'VENT_YN', 'MDSCO_PRTC_YN', 'NATN_NM']

for df in [df_rest_select, df_active_select]:
    for feature in encode_features:
        le = LabelEncoder()
        le = le.fit(df[feature])
        df[feature] = le.transform(df[feature])

df_active_select

,BIZ_NO,BZ_TYP,NATN_NM,PBCO_GB,PSN_CORP_GB,HDOF_BR_GB,FR_IVST_CORP_YN,VENT_YN,LIST_CD,IND_CD_ORDR,IND_CD1,IND_CD2,IND_CD3,MDSCO_PRTC_YN,HDOF_CMP_CD,ESTB_DATE,ESTB_GB,LIST_DATE,EMP_CNT
0,1138111739,M,0,2,1,1,0,0,NaN,10,47712,NaN,NaN,0,NaN,19660419,01,NaN,43
1,2078132193,M,0,2,1,1,0,0,NaN,10,15190,NaN,NaN,0,NaN,19910806,01,NaN,35
2,1318167565,M,0,2,1,1,0,0,NaN,10,49212,NaN,NaN,0,NaN,20011030,01,NaN,149
3,5048128251,M,0,2,1,1,0,0,NaN,10,30391,NaN,NaN,0,NaN,19990112,01,NaN,75
4,1098164260,M,0,2,1,1,0,0,NaN,10,46800,NaN,NaN,0,NaN,20000531,01,NaN,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28977,8048101453,M,0,2,1,1,0,0,NaN,NaN,NaN,NaN,NaN,0,NaN,20191206,01,NaN,0
28978,8028701430,M,0,2,1,1,0,0,NaN,10,68122,NaN,NaN,0,NaN,20200402,01,NaN,1
28979,4268501421,M,0,2,1,2,0,0,NaN,10,72100,NaN,NaN,0,1220726,19900108,01,NaN,81
28980,6518502462,M,0,2,1,2,0,0,NaN,10,23322,NaN,NaN,0,1583116,20220615,01,NaN,26


In [17]:
#국가(NATN_NM) 갯수
countryDF = pd.DataFrame(df_active_select['NATN_NM'].value_counts())
country_list=countryDF.index.tolist()
len(country_list)
#국가를 대륙별로 모을까.?? -> 일단 라벨인코딩 완료

64

In [18]:
#원핫인코딩
#PBCO_GB	공기업구분 1.공기업 2.일반기업
# PSN_CORP_GB	개인법인구분 1.법인 2.개인
# HDOF_BR_GB	본점지점구분 1.본점 2.지점

#하나의 컬럼마다 2차원데이터프레임으로 존재해야해서 원핫인코딩 불가!
#라벨인코딩도 굳이 안해도 될 듯해서 일단 pass


In [19]:
#LIST_CD 상장코드 - 상장유무로 0,1로 바꾸기

df_active_select['LIST_CD'] = df_active_select['LIST_CD'].replace('NaN',0)
df_rest_select['LIST_CD'] = df_rest_select['LIST_CD'].replace('NaN',0)

df_rest_select

,BIZ_NO,BZ_TYP,NATN_NM,PBCO_GB,PSN_CORP_GB,HDOF_BR_GB,FR_IVST_CORP_YN,VENT_YN,LIST_CD,IND_CD_ORDR,IND_CD1,IND_CD2,IND_CD3,MDSCO_PRTC_YN,HDOF_CMP_CD,ESTB_DATE,ESTB_GB,LIST_DATE,EMP_CNT
0,1268145633,M,0,2,1,1,0,0,0,10,30391,NaN,NaN,0,NaN,20000529,01,NaN,40
1,1388101749,M,0,2,1,1,0,0,0,10,46610,NaN,NaN,0,NaN,19850110,01,NaN,13
2,1068179240,M,0,2,1,1,0,0,0,10,46510,NaN,NaN,0,NaN,19990601,01,NaN,1
3,2248114507,M,0,2,1,1,0,0,0,10,25991,NaN,NaN,0,NaN,19980522,01,NaN,6
4,1168132785,M,0,2,1,1,0,0,0,10,24199,NaN,NaN,0,NaN,19930226,01,NaN,40
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6734,4808501629,M,0,2,1,2,0,0,0,10,10200,NaN,NaN,0,8474403,20180126,01,NaN,NaN
6735,8018501578,M,0,2,1,2,0,0,0,10,10122,NaN,NaN,0,4033821,20200727,01,NaN,22
6736,4118523419,M,0,2,1,2,0,0,0,10,56111,56121,NaN,0,9189675,20190926,01,NaN,0
6737,1738102533,M,0,2,1,1,0,0,0,10,58222,NaN,NaN,0,NaN,20211025,01,NaN,NaN


In [20]:
#액티브에서 상장기업 89개
LISTCD = df_active_select.loc[df_active_select['LIST_CD']!=0]
len(LISTCD)

89

In [21]:
#휴업에서 상장기업 개수 / 비상장6693
LISTCD2 = df_rest_select.loc[df_rest_select['LIST_CD']!=0]
len(LISTCD2)

46

In [22]:
#상장기업인지 아닌지 컬럼 하나 추가
df_active_select['Listing'] = df_active_select['LIST_CD']
df_rest_select['Listing'] = df_rest_select['LIST_CD']

#조건식을 따로 작성!!!
listing_condition = (df_active_select['Listing'] != 0)
listing_condition2 = (df_rest_select['Listing'] != 0)

df_active_select.loc[listing_condition, 'Listing']=1
df_rest_select.loc[listing_condition2, 'Listing']=1

df_rest_select.tail(30)



,BIZ_NO,BZ_TYP,NATN_NM,PBCO_GB,PSN_CORP_GB,HDOF_BR_GB,FR_IVST_CORP_YN,VENT_YN,LIST_CD,IND_CD_ORDR,IND_CD1,IND_CD2,IND_CD3,MDSCO_PRTC_YN,HDOF_CMP_CD,ESTB_DATE,ESTB_GB,LIST_DATE,EMP_CNT,Listing
6709,2028166698,M,0,2,1,1,0,1,0,10,26299,NaN,NaN,0,NaN,20200724,01,NaN,30,0
6710,4328101957,M,0,2,1,1,0,0,0,10,13229,NaN,NaN,0,NaN,20200721,01,NaN,NaN,0
6711,3568800162,M,0,2,1,1,0,0,0,10,58221,62010,NaN,0,NaN,20151027,01,NaN,17,0
6712,1818102010,M,0,2,1,1,0,0,0,10,41229,NaN,NaN,0,NaN,20200812,01,NaN,6,0
6713,2588801737,M,0,2,1,1,0,0,0,10,10802,NaN,NaN,0,NaN,20200810,01,NaN,0,0
6714,7118801975,M,0,2,1,1,0,1,0,10,58222,NaN,NaN,0,NaN,20200813,01,NaN,NaN,0
6715,8118801810,M,0,2,1,1,0,0,0,10,41229,NaN,NaN,0,NaN,20200813,01,NaN,5,0
6716,6018701900,M,0,2,1,1,0,0,0,10,23900,NaN,NaN,0,NaN,20200901,01,NaN,114,0
6717,1908101790,M,0,2,1,1,0,0,0,10,41229,NaN,NaN,0,NaN,20200828,01,NaN,7,0
6718,7268602012,M,0,2,1,1,0,0,0,10,58220,NaN,NaN,0,NaN,20200904,01,NaN,16,0


In [23]:
df_rest_select.groupby('Listing').count()

,BIZ_NO,BZ_TYP,NATN_NM,PBCO_GB,PSN_CORP_GB,HDOF_BR_GB,FR_IVST_CORP_YN,VENT_YN,LIST_CD,IND_CD_ORDR,IND_CD1,IND_CD2,IND_CD3,MDSCO_PRTC_YN,HDOF_CMP_CD,ESTB_DATE,ESTB_GB,LIST_DATE,EMP_CNT
Listing,,,,,,,,,,,,,,,,,,,
0,6693,6693,6693,6693,6693,6693,6693,6693,6693,6693,6693,6693,6693,6693,6693,6693,6693,6693,6693
1,46,46,46,46,46,46,46,46,46,46,46,46,46,46,46,46,46,46,46


In [24]:
df_active_select.dtypes

0
BIZ_NO             object
BZ_TYP             object
NATN_NM             int32
PBCO_GB            object
PSN_CORP_GB        object
HDOF_BR_GB         object
FR_IVST_CORP_YN     int32
VENT_YN             int32
LIST_CD            object
IND_CD_ORDR        object
IND_CD1            object
IND_CD2            object
IND_CD3            object
MDSCO_PRTC_YN       int32
HDOF_CMP_CD        object
ESTB_DATE          object
ESTB_GB            object
LIST_DATE          object
EMP_CNT            object
Listing            object
dtype: object

In [25]:
#type을 float로 바꿀 컬럼들
float_columns = ['NATN_NM', 'PBCO_GB', 'PSN_CORP_GB', 'HDOF_BR_GB', 'FR_IVST_CORP_YN', 'VENT_YN', 'MDSCO_PRTC_YN', 'ESTB_GB']

#NaN값 있어서 못바꾸는 것들 EMP_CNT, IND_CD_ORDR
#직원수 NaN값 뭐할지 정하기 - 평균값?

In [26]:
#type 변경
for columns in float_columns:
    df_active_select[columns] = pd.to_numeric(df_active_select[columns])
    df_rest_select[columns] = pd.to_numeric(df_rest_select[columns])

In [27]:
df_rest_select.dtypes

0
BIZ_NO             object
BZ_TYP             object
NATN_NM             int32
PBCO_GB             int64
PSN_CORP_GB         int64
HDOF_BR_GB          int64
FR_IVST_CORP_YN     int32
VENT_YN             int32
LIST_CD            object
IND_CD_ORDR        object
IND_CD1            object
IND_CD2            object
IND_CD3            object
MDSCO_PRTC_YN       int32
HDOF_CMP_CD        object
ESTB_DATE          object
ESTB_GB             int64
LIST_DATE          object
EMP_CNT            object
Listing            object
dtype: object

In [28]:
#직원수의 전체 데이터 요약 (중간값, 평균값, min, max) 
df_active_select['EMP_CNT'].describe()

count     28982
unique      747
top           1
freq       1341
Name: EMP_CNT, dtype: int64

In [29]:
df_rest_select['EMP_CNT'].describe()

count     6739
unique     246
top        NaN
freq       688
Name: EMP_CNT, dtype: object

In [31]:
#구간화 - 직원수EMP_CNT	, 설립일자ESTB_DATE
#그룹화? - BZ_TYP, IND_CD1	
#빼고 완료

In [30]:
df_active_record

,BIZ_NO,END_DATE,STRT_DATE,CLSBZ_GB,STAT_OCR_DATE
0,5088115592,99991231,20150209,5,NaN
1,5088115592,20150208,20200731,2,NaN
2,1138127100,99991231,20121105,5,NaN
3,1138127100,20121104,20200731,2,NaN
4,1138127100,20121004,20200731,5,NaN
...,...,...,...,...,...
4822,6628701486,20220913,20220830,2,NaN
4823,6098801857,99991231,20220921,6,NaN
4824,6098801857,20220920,20220607,3,20220607
4825,4988102183,99991231,20220705,5,NaN


In [31]:
df_active_record['CLSBZ_GB'] = pd.to_numeric(df_active_record['CLSBZ_GB'])
df_rest_record['CLSBZ_GB'] = pd.to_numeric(df_rest_record['CLSBZ_GB'])
df_rest_record

,BIZ_NO,END_DATE,STRT_DATE,CLSBZ_GB,STAT_OCR_DATE
0,1268145633,99991231,20220518,3,20220518
1,1388101749,99991231,20210319,3,20210319
2,1068179240,99991231,20200731,3,20191213
3,2248114507,99991231,20210531,3,20210531
4,1168132785,99991231,20210630,3,20210630
...,...,...,...,...,...
6835,4808501629,99991231,20211231,3,20211231
6836,8018501578,99991231,20211126,3,20211126
6837,4118523419,99991231,20220630,3,20220630
6838,1738102533,99991231,20220731,3,20220731


In [32]:
#close_y_n : 휴폐업 유무, close:휴폐업 횟수
df_rest_record['close_y_n']=1
df_active_record['close_y_n']=1
df_active_record


,BIZ_NO,END_DATE,STRT_DATE,CLSBZ_GB,STAT_OCR_DATE,close_y_n
0,5088115592,99991231,20150209,5,NaN,1
1,5088115592,20150208,20200731,2,NaN,1
2,1138127100,99991231,20121105,5,NaN,1
3,1138127100,20121104,20200731,2,NaN,1
4,1138127100,20121004,20200731,5,NaN,1
...,...,...,...,...,...,...
4822,6628701486,20220913,20220830,2,NaN,1
4823,6098801857,99991231,20220921,6,NaN,1
4824,6098801857,20220920,20220607,3,20220607,1
4825,4988102183,99991231,20220705,5,NaN,1


In [33]:
#휴폐업이력sheet에 있는 사업자등록번호랑 연결해서
#단순하게 2,3번을 count해서 휴폐업 횟수 구해서 close_count컬럼만들기

#사업자등록번호 겹치는거 다 하나씩만 남기고, 새 컬럼에 2,3번 count하나 만들어서 값 넣고
#join(outer)로 BIZ_NO로 연결해서 df만들기


#set인덱스한 df
df_active_record=df_active_record.set_index('BIZ_NO')
df_rest_record=df_rest_record.set_index('BIZ_NO')
df_rest_record



,END_DATE,STRT_DATE,CLSBZ_GB,STAT_OCR_DATE,close_y_n
BIZ_NO,,,,,
1268145633,99991231,20220518,3,20220518,1
1388101749,99991231,20210319,3,20210319,1
1068179240,99991231,20200731,3,20191213,1
2248114507,99991231,20210531,3,20210531,1
1168132785,99991231,20210630,3,20210630,1
...,...,...,...,...,...
4808501629,99991231,20211231,3,20211231,1
8018501578,99991231,20211126,3,20211126,1
4118523419,99991231,20220630,3,20220630,1


In [34]:
#휴폐업 조건으로 필터링한 새로운 컬럼 - 휴폐업했던 애들만 들어있는 df
close_condition = (df_active_record.CLSBZ_GB==1) | (df_active_record.CLSBZ_GB==2) | (df_active_record.CLSBZ_GB==3)
close_condition2 = (df_rest_record.CLSBZ_GB==1) | (df_rest_record.CLSBZ_GB==2) | (df_rest_record.CLSBZ_GB==3)
condition_df_active_record = df_active_record[close_condition]
condition_df_rest_record = df_rest_record[close_condition2]

condition_df_active_record

,END_DATE,STRT_DATE,CLSBZ_GB,STAT_OCR_DATE,close_y_n
BIZ_NO,,,,,
5088115592,20150208,20200731,2,NaN,1
1138127100,20121104,20200731,2,NaN,1
1138127100,20100726,20200731,2,NaN,1
1348608497,20131027,20200731,2,NaN,1
1348608497,20101031,20200731,2,NaN,1
...,...,...,...,...,...
3628601745,20211108,20211102,2,NaN,1
4888601765,20211011,20210707,2,NaN,1
6628701486,20220913,20220830,2,NaN,1


In [35]:
# condition_df_active_record=condition_df_active_record.set_index('BIZ_NO')
active_r_c = condition_df_active_record.groupby('BIZ_NO').sum()
rest_r_c = condition_df_rest_record.groupby('BIZ_NO').sum()
rest_r_c['close_y_n']

BIZ_NO
1018135422    1
1018154206    1
1018163684    1
1018178760    1
1018194173    1
             ..
8988100371    1
8988600037    1
8998600312    1
8998700283    1
8998701263    1
Name: close_y_n, Length: 6746, dtype: int64

In [36]:
df_active_record['close_count'] = active_r_c['close_y_n']
df_rest_record['close_count'] = rest_r_c['close_y_n']
df_rest_record

#close_count가 NaN값으로 자꾸 들어감.. set_index 안하고 하니까 해서 그런가? 
#set index하고 똑같이 하니까 되네. 무슨 차이지..

,END_DATE,STRT_DATE,CLSBZ_GB,STAT_OCR_DATE,close_y_n,close_count
BIZ_NO,,,,,,
1268145633,99991231,20220518,3,20220518,1,1
1388101749,99991231,20210319,3,20210319,1,1
1068179240,99991231,20200731,3,20191213,1,1
2248114507,99991231,20210531,3,20210531,1,1
1168132785,99991231,20210630,3,20210630,1,1
...,...,...,...,...,...,...
4808501629,99991231,20211231,3,20211231,1,1
8018501578,99991231,20211126,3,20211126,1,1
4118523419,99991231,20220630,3,20220630,1,1


In [37]:
df_active_record=df_active_record.reset_index()
df_rest_record=df_rest_record.reset_index()
df_active_record

,BIZ_NO,END_DATE,STRT_DATE,CLSBZ_GB,STAT_OCR_DATE,close_y_n,close_count
0,5088115592,99991231,20150209,5,NaN,1,1.0
1,5088115592,20150208,20200731,2,NaN,1,1.0
2,1138127100,99991231,20121105,5,NaN,1,2.0
3,1138127100,20121104,20200731,2,NaN,1,2.0
4,1138127100,20121004,20200731,5,NaN,1,2.0
...,...,...,...,...,...,...,...
4822,6628701486,20220913,20220830,2,NaN,1,1.0
4823,6098801857,99991231,20220921,6,NaN,1,1.0
4824,6098801857,20220920,20220607,3,20220607,1,1.0
4825,4988102183,99991231,20220705,5,NaN,1,1.0


In [38]:
df_active_record['BIZ_NO'] = pd.to_numeric(df_active_record['BIZ_NO'])
df_rest_record['BIZ_NO'] = pd.to_numeric(df_rest_record['BIZ_NO'])

In [39]:
#중복인덱스 삭제하기 - 열지정하여 중복제거
df_active_record= df_active_record.drop_duplicates(['BIZ_NO']).reset_index(drop=True)
df_rest_record= df_rest_record.drop_duplicates(['BIZ_NO']).reset_index(drop=True)

In [40]:
df_active_record

,BIZ_NO,END_DATE,STRT_DATE,CLSBZ_GB,STAT_OCR_DATE,close_y_n,close_count
0,5088115592,99991231,20150209,5,NaN,1,1.0
1,1138127100,99991231,20121105,5,NaN,1,2.0
2,1348608497,99991231,20200731,5,NaN,1,3.0
3,1148193374,99991231,20190312,5,NaN,1,1.0
4,4098167376,99991231,20200731,5,NaN,1,1.0
...,...,...,...,...,...,...,...
2078,3628601745,99991231,20211109,5,NaN,1,1.0
2079,4888601765,99991231,20211012,5,NaN,1,1.0
2080,6628701486,99991231,20220914,5,NaN,1,1.0
2081,6098801857,99991231,20220921,6,NaN,1,1.0


In [41]:
df_active_select

,BIZ_NO,BZ_TYP,NATN_NM,PBCO_GB,PSN_CORP_GB,HDOF_BR_GB,FR_IVST_CORP_YN,VENT_YN,LIST_CD,IND_CD_ORDR,IND_CD1,IND_CD2,IND_CD3,MDSCO_PRTC_YN,HDOF_CMP_CD,ESTB_DATE,ESTB_GB,LIST_DATE,EMP_CNT,Listing
0,1138111739,M,0,2,1,1,0,0,0,10,47712,NaN,NaN,0,NaN,19660419,1,NaN,43,0
1,2078132193,M,0,2,1,1,0,0,0,10,15190,NaN,NaN,0,NaN,19910806,1,NaN,35,0
2,1318167565,M,0,2,1,1,0,0,0,10,49212,NaN,NaN,0,NaN,20011030,1,NaN,149,0
3,5048128251,M,0,2,1,1,0,0,0,10,30391,NaN,NaN,0,NaN,19990112,1,NaN,75,0
4,1098164260,M,0,2,1,1,0,0,0,10,46800,NaN,NaN,0,NaN,20000531,1,NaN,3,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28977,8048101453,M,0,2,1,1,0,0,0,NaN,NaN,NaN,NaN,0,NaN,20191206,1,NaN,0,0
28978,8028701430,M,0,2,1,1,0,0,0,10,68122,NaN,NaN,0,NaN,20200402,1,NaN,1,0
28979,4268501421,M,0,2,1,2,0,0,0,10,72100,NaN,NaN,0,1220726,19900108,1,NaN,81,0
28980,6518502462,M,0,2,1,2,0,0,0,10,23322,NaN,NaN,0,1583116,20220615,1,NaN,26,0


In [42]:
len(df_active_select)

28982

In [75]:
test = pd.DataFrame(sheet_active.values)
test.columns = test.iloc[0, :]
test = test.iloc[1:, :]
test = test.reset_index(drop=True)
test = test['ESTB_DATE']
type(test)

pandas.core.series.Series

In [79]:
test = pd.DataFrame(test)
test['ESTB_DATE']=test['ESTB_DATE'][-2:]


In [80]:
test

,ESTB_DATE
0,NaN
1,NaN
2,NaN
3,NaN
4,NaN
...,...
28977,NaN
28978,NaN
28979,NaN
28980,20220615


In [78]:
pd.to_datetime(test['ESTB_DATE'])

ParserError: day is out of range for month: 20000700

In [70]:
try:
    test['estb_date'] =pd.to_datetime(test['ESTB_DATE'])
except:
    test['estb_date']='NaN'
test.groupby('estb_date').count()

,BIZ_NO,CMP_PFIX_NM,CMP_NM1,CMP_SFIX_NM,CMP_ENM,BZ_TYP,CMP_SCL,PBCO_GB,PSN_CORP_GB,HDOF_BR_GB,...,MDSCO_PRTC_YN,HDOF_CMP_CD,ESTB_DATE,ESTB_GB,LIST_DATE,MN_BIZ_CONT,NATN_NM,HOMEPAGE_URL,CEO_NM,EMP_CNT
estb_date,,,,,,,,,,,,,,,,,,,,,
NaN,28982,18580,28982,9169,28965,28982,28982,28982,28982,28982,...,28982,28982,27908,28982,159,28819,1645,17765,28981,28982


In [71]:
test

,BIZ_NO,CMP_PFIX_NM,CMP_NM1,CMP_SFIX_NM,CMP_ENM,BZ_TYP,CMP_SCL,PBCO_GB,PSN_CORP_GB,HDOF_BR_GB,...,HDOF_CMP_CD,ESTB_DATE,ESTB_GB,LIST_DATE,MN_BIZ_CONT,NATN_NM,HOMEPAGE_URL,CEO_NM,EMP_CNT,estb_date
0,1138111739,None,한일가스산업,(주),"HANIL GAS IND. CO.,LTD.",M,2,2,1,1,...,,19660419,01,None,LP Gas 충전 및 판매업,None,www.hanilgas.com,홍순철외 1명,43,NaN
1,2078132193,(주),엠피아이,None,"EMPIALEE CO.,LTD.",M,2,2,1,1,...,,19910806,01,None,가죽 및 대용가죽제품 제조업,None,www.mastercnd.co.kr,이스테판상수외 1명,35,NaN
2,1318167565,None,도영운수,(주),"DOYOUNG TRANSPORTATION CO.,LTD.",M,2,2,1,1,...,,20011030,01,None,시내버스업,None,transport@naver.com,김명화,149,NaN
3,5048128251,(주),중원산업,None,"JOONGWON INDUSTRIAL CO.,LTD.",M,2,2,1,1,...,,19990112,01,None,자동차 부품 제조,None,www.koreajw.com,김명구,75,NaN
4,1098164260,(주),카타나골프,None,"CATANA GOLF CO.,LTD.",M,2,2,1,1,...,,20000531,01,None,골프용품 전반의 제조 및 수출업,None,None,김홍득,3,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28977,8048101453,None,제이앤인베스트먼트,None,J&INVESTMENT,M,2,2,1,1,...,,20191206,01,None,경영컨설팅업,None,None,정경태,0,NaN
28978,8028701430,None,디앤엠개발,(주),"DNM Development Co., Ltd.",M,2,2,1,1,...,,20200402,01,None,비주거용 건물 임대업,None,None,송성현,1,NaN
28979,4268501421,None,마스비 건축연구소,None,None,M,2,2,1,2,...,1220726,19900108,01,None,"BIM용역, 엔지니어링컨설팅",None,None,김태남,81,NaN
28980,6518502462,(주),건설레미콘의정부공장,None,"Construction Remicon Uijeongbu Factory Co., Ltd.",M,2,2,1,2,...,1583116,20220615,01,None,레미콘,None,None,이미성,26,NaN


In [195]:
#df_active_select, df_rest_select에 업력 컬럼 만들기
#업력=설립날짜-오늘날짜   (휴업기간는 고려x) ESTB_DATE



now=datetime.now()

for df in [df_active_select, df_rest_select]:
    #리스트로 만들어서 .. 하나씩 변형해서 또 리스트로 만들어서 컬럼에 넣을까..... 
    df['estb_date'] =pd.to_datetime(df['ESTB_DATE'])
    df['history']= (now - df['estb_date']).days //365




# history = (now-estb_date)
# his_day = history.days
# his_year= (his_day)//365


# date_establish = datetime.strptime(컬럼명,"%Y%m%d")
# 업력기간=(now-date_establish-휴업기간).days
# 업력=업력기간//365 


TypeError: strptime() argument 1 must be str, not Series

In [ ]:

#중복인덱스 제거한 다음에 merge나 join해서 데이터프레임 사업자번호기준으로 합치기


In [ ]:
문제
# 1.휴폐업이력에서 휴페업발생일자가 존재하지 않는 기업이 너무 많음 
# -> 휴업기간을 구할 수 없어서 업력을 구하기 애매
# => 일단 업력을 단순히 연도 해보기 -> 연도로 해야되나 아니면 햇수로..?


3.정답(y)컬럼 만들기
한번이라도 했으면 1값, 아니면 0


4.재무데이터 널값처리 - 12개

5.데이터 불균형 맞춰주기?

#직원수 NaN값 뭐할지 정하기 - 평균값?

In [ ]:
#휴폐업 정답컬럼추가 - 0/1


#정답(y)컬럼을 만들때, 한번이라도 휴폐업한 기업이면 1값?
액티브+휴폐업경험
액티브+휴폐업없었음
현재휴폐업

## 시각화

In [160]:
#type을 float로 바꾼 컬럼들
float_columns = ['NATN_NM', 'PBCO_GB', 'PSN_CORP_GB', 'HDOF_BR_GB', 'FR_IVST_CORP_YN', 'VENT_YN', 'IND_CD_ORDR', 'MDSCO_PRTC_YN', 'ESTB_GB', 'EMP_CNT']

Index(['BIZ_NO', 'BZ_TYP', 'NATN_NM', 'PBCO_GB', 'PSN_CORP_GB', 'HDOF_BR_GB',
       'FR_IVST_CORP_YN', 'VENT_YN', 'LIST_CD', 'IND_CD_ORDR', 'IND_CD1',
       'IND_CD2', 'IND_CD3', 'MDSCO_PRTC_YN', 'HDOF_CMP_CD', 'ESTB_DATE',
       'ESTB_GB', 'LIST_DATE', 'EMP_CNT', 'Listing'],
      dtype='object', name=0)